In [22]:
# Imports

import numpy as np
import pandas as pd
from operator import mod 
import streamlit as st 
import pickle 
import os # need for file enumeration

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import utils

# For reproducibility
np.random.seed(42)

In [6]:
# get our file listings for train and test
train_files = os.listdir('images/train/')
test_files = os.listdir('images/test/')

In [7]:
# convert filenames into dataframes
train_df = pd.DataFrame(train_files, columns = ['filename'])
test_df = pd.DataFrame(test_files, columns = ['filename'])

In [8]:
# label 'em
train_df['hotdog'] = train_df['filename'].map(lambda x:
    'hotdog' if x.startswith('hotdog') else 'nothotdog')
test_df['hotdog'] = test_df['filename'].map(lambda x:
    'hotdog' if x.startswith('hotdog') else 'nothotdog')

In [15]:
# set up imagedatagenerators
from turtle import colormode


train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_dataframe(
        dataframe = train_df,
        directory = 'images/train',
        x_col = 'filename',
        y_col = 'hotdog',
        target_size=(300, 300),
        batch_size=32,
        class_mode='binary',
        color_mode = 'grayscale')
validation_generator = test_datagen.flow_from_dataframe(
        dataframe = test_df,
        directory = 'images/test',
        x_col = 'filename',
        y_col = 'hotdog',
        target_size=(300, 300),
        batch_size=32,
        class_mode='binary',
        color_mode = 'grayscale')

Found 3000 validated image filenames belonging to 2 classes.
Found 644 validated image filenames belonging to 2 classes.


In [16]:
# train baseline
train_df['hotdog'].value_counts(normalize = True)

hotdog       0.5
nothotdog    0.5
Name: hotdog, dtype: float64

In [17]:
# test baseline
test_df['hotdog'].value_counts(normalize = True)

nothotdog    0.5
hotdog       0.5
Name: hotdog, dtype: float64

In [19]:
# set up model
cnn_model = Sequential()

cnn_model.add(Conv2D( 
  filters = 6, # number of filters
  kernel_size = (3, 3), # height/width of filter
  activation = 'relu', # activation function 
  input_shape = (300, 300, 3) # shape of input (image)
))

cnn_model.add(MaxPooling2D(pool_size = (2, 2))) # dimensions of region of pooling

cnn_model.add(Conv2D(filters = 16,
                     kernel_size = (3, 3),
                     activation = 'relu'))

cnn_model.add(MaxPooling2D(pool_size = (2, 2)))

cnn_model.add(Flatten())

cnn_model.add(Dense(units = 128, activation = 'relu'))

cnn_model.add(Dense(units = 1, activation = 'sigmoid'))

cnn_model.compile(loss = 'binary_crossentropy',
                  optimizer ='adam',
                  metrics = ['accuracy'])

In [21]:
# fit model
cnn_model.fit(
        train_generator,
        epochs = 10,
        validation_data = validation_generator,
        batch_size = 128)

Epoch 1/10
94/94 [==============================] - 59s 629ms/step - loss: 0.6378 - accuracy: 0.6343 - val_loss: 0.7011 - val_accuracy: 0.6040
Epoch 2/10
94/94 [==============================] - 58s 618ms/step - loss: 0.6206 - accuracy: 0.6583 - val_loss: 0.7110 - val_accuracy: 0.6304
Epoch 3/10
94/94 [==============================] - 61s 642ms/step - loss: 0.6089 - accuracy: 0.6663 - val_loss: 0.6372 - val_accuracy: 0.6832
Epoch 4/10
94/94 [==============================] - 56s 589ms/step - loss: 0.6028 - accuracy: 0.6660 - val_loss: 0.6983 - val_accuracy: 0.6289
Epoch 5/10
94/94 [==============================] - 56s 596ms/step - loss: 0.5988 - accuracy: 0.6753 - val_loss: 0.6134 - val_accuracy: 0.6863
Epoch 6/10
94/94 [==============================] - 59s 627ms/step - loss: 0.6015 - accuracy: 0.6647 - val_loss: 0.7833 - val_accuracy: 0.6413
Epoch 7/10
94/94 [==============================] - 59s 622ms/step - loss: 0.5744 - accuracy: 0.6970 - val_loss: 0.7991 - val_accuracy: 0.6258

In [27]:
cnn_model.save('./models/cnn_1_exported')

2022-10-21 13:30:01.076 Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 2 of 2). These functions will not be directly callable after loading.
2022-10-21 13:30:02.042 Assets written to: ./models/cnn_1_exported/assets
